# Generation of files for media preparation using biomek

This notebook generates several files used in an automated media optimization project. Given a file with stock concentrations and a file with target concentrations (recommended for example by ART), this notebooks generates transfer volumes needed to achieve those target concentrations. Transfer volumes are provided as a table for all components and destination wells, but also as files required for a Biomek run. The notebook also generates a file with instructions on how to prepare a plate with stock solutions.

## Inputs and outputs

### Required files to run this notebook:
   - `standard_recipe_concentrations.csv` - this file will not change over the course of a particular project
   
   - `stock_concentrations.csv` - this file will not change over the course of a particular project
   
   - `../data/DBTLX/target_concentrations.csv` - this is an output from an ART run and it will change at every DBTL cycle
   
   - `24-well_stock_plate_high.csv`, `24-well_stock_plate_low.csv`, `24-well_stock_plate_fresh.csv` - instructions on how to prepare the source plates
     

### Files generated by running this notebook:
   
   - `dest_volumes.csv` - volumes of all components in each of the destination wells
   
   - `media_descriptions.csv` - complete list of media designs in terms of concentrations for each each well
   
   
   
The files that need to be uploaded to the Biomek, following this particular order:
   
   - `P200_water.csv`
   
   - `P20_water.csv`
         
   - `P20_kan.csv` 
   
   - `P200_components.csv` 
   
   - `P20_components.csv` 
  
   - `P20_culture.csv` 
   
   
Note that all these different files are needed as biomek needs a separate one for different types of source/destination plate and tips. Also, it cannot follow the order of operations within a file so we have to enforce an order using different files (e.g. first a file with kan transfers, then a file with components transfers).
    
The files are stored in the user defined directory. 

## Setup

Importing needed libraries:

In [1]:
import os
import sys
sys.path.append('../')

import pandas as pd
import numpy as np

from core import find_volumes, find_volumes_bulk


## User parameters

In [2]:
CYCLE = 6

user_params = {
    'stock_conc_file': '../data/flaviolin/stock_concentrations.csv',
    'standard_media_file': '../data/flaviolin/standard_recipe_concentrations.csv',
    'stock_plate_high_file': '../data/flaviolin/24-well_stock_plate_high.csv',
    'stock_plate_low_file': '../data/flaviolin/24-well_stock_plate_low.csv',
    'stock_plate_fresh_file': '../data/flaviolin/24-well_stock_plate_fresh.csv',
    'target_conc_file': f'../data/flaviolin/DBTL{CYCLE}/target_concentrations.csv',
    'output_path': f'../data/flaviolin/DBTL{CYCLE}',  # Path for output files
    'well_volume': 1500,            # Total volume of the media content in the destination well
    'tips': ['f20', 'f200'],        # Choose available tips from f20, s20, f50, s50, f200, s200
    'min_transfer_volume': 5.,      # Minimal transfer volume of the liquid handler
    'culture_factor': 100           # Dilution factor for culture, e.g. 100x, 1000x

}

Setup tips

In [3]:
df_tips = pd.DataFrame([['f20', 36],
                        ['s20', 72],
                        ['f50', 45],
                        ['s50', 81],
                        ['f200', 190],
                        ['s200', 171]],
                       columns=['Tips', 'Max Volume'],
                       ).set_index('Tips')


In [4]:
max_tip_volume = max([df_tips.loc[tip]['Max Volume'] for tip in user_params['tips']])


Load the standard media recipe

In [5]:
df_stand = pd.read_csv(user_params['standard_media_file']).set_index("Component")

In [6]:
# df_stand

Load the stock concentrations and stock plate layouts:

In [7]:
df_stock = pd.read_csv(user_params['stock_conc_file'])
df_stock = df_stock.set_index("Component")
df_stock_plate_high = pd.read_csv(user_params['stock_plate_high_file'])
df_stock_plate_low = pd.read_csv(user_params['stock_plate_low_file'])
df_stock_plate_fresh = pd.read_csv(user_params['stock_plate_fresh_file'])


In [8]:
df_stock_plate_fresh

,Well,Component,Concentration[mM]
0,A1,Culture,NaN
1,B1,FeSO4,0.3
2,C1,FeSO4,6.0


Reformat values from string to float in the `df_stock` dataframe:

In [9]:
df_stock.loc['Kan'] = [300., 300., 1.]
df_stock["High Concentration[mM]"] = df_stock["High Concentration[mM]"].astype(float)
df_stock["Low Concentration[mM]"] = df_stock["Low Concentration[mM]"].astype(float)

In [10]:
df_stock

,Low Concentration[mM],High Concentration[mM],Dilution Factor
Component,,,
MOPS,2000.0000,2000.0000,1.0
Tricine,400.0000,400.0000,1.0
H3BO3,0.1200,2.4000,20.0
Glucose,3000.0000,3000.0000,1.0
K2SO4,8.7000,43.5000,5.0
K2HPO4,79.2000,396.0000,5.0
FeSO4,0.3000,6.0000,20.0
NH4Cl,1904.0001,1904.0001,1.0
MgCl2,15.6000,15.6000,1.0


## Read target concentrations

Read ART suggested target concentrations. Note that those are only for components which are being explored (not those that are kept constant). 

In [11]:
target_conc_file = user_params['target_conc_file']
df_target_conc = pd.read_csv(target_conc_file, index_col=0)
df_target_conc.head()

,H3BO3,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4
Well,,,,,,,,,,,,
A1,0.025429,2.356942,3.646289,0.007856,6.680308,4.907831,15.155637,0.000093,0.000721,0.000950,0.004621,0.000744
B1,0.025429,2.356942,3.646289,0.007856,6.680308,4.907831,15.155637,0.000093,0.000721,0.000950,0.004621,0.000744
C1,0.025429,2.356942,3.646289,0.007856,6.680308,4.907831,15.155637,0.000093,0.000721,0.000950,0.004621,0.000744
D1,0.008628,0.896852,5.044414,0.017022,13.269195,2.521720,232.495019,0.000104,0.002004,0.000558,0.001644,0.000029
E1,0.008628,0.896852,5.044414,0.017022,13.269195,2.521720,232.495019,0.000104,0.002004,0.000558,0.001644,0.000029


### Add fixed components and antibiotic concentrations from the standard recipe to the target concentrations dataframe

Find fixed components as those from the standard recipe that are not listed in target concentration file, which is an output from ART

In [12]:
comp_fixed = list(df_stand.drop(df_target_conc.columns).index)
print('Fixed components: ')
for comp in comp_fixed:
    df_target_conc[comp] = df_stand.at[comp, 'Concentration[mM]']
    print(f'{comp}')

Fixed components: 
MOPS
Tricine
Glucose
Kan


Make sure the order of columns is the same as in the stock dataframe:

In [13]:
columns = df_stock.index
df_target_conc = df_target_conc[columns]

In [14]:
df_target_conc.head()

,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Kan
Well,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.025429,20.0,2.356942,3.646289,0.007856,6.680308,4.907831,15.155637,0.000093,0.000721,0.000950,0.004621,0.000744,1.0
B1,40.0,4.0,0.025429,20.0,2.356942,3.646289,0.007856,6.680308,4.907831,15.155637,0.000093,0.000721,0.000950,0.004621,0.000744,1.0
C1,40.0,4.0,0.025429,20.0,2.356942,3.646289,0.007856,6.680308,4.907831,15.155637,0.000093,0.000721,0.000950,0.004621,0.000744,1.0
D1,40.0,4.0,0.008628,20.0,0.896852,5.044414,0.017022,13.269195,2.521720,232.495019,0.000104,0.002004,0.000558,0.001644,0.000029,1.0
E1,40.0,4.0,0.008628,20.0,0.896852,5.044414,0.017022,13.269195,2.521720,232.495019,0.000104,0.002004,0.000558,0.001644,0.000029,1.0


Save a file with a complete list of components concentrations, describing the media (this file will lated be used to create experiment description file for EDD import).

In [15]:
final_conc_file = f"{user_params['output_path']}/media_descriptions.csv"
df_target_conc.drop(columns='Kan').to_csv(final_conc_file)

## Calculate all transfer volumes

Also, create a dataframe with levels of stock concentrations needed to achieve those volumes, which will indicate from which source well the transfer should be made.

In [16]:
df_volumes, df_conc_level = find_volumes_bulk(
    df_stock=df_stock, 
    df_target_conc=df_target_conc,
    well_volume=user_params['well_volume'],
    min_tip_volume=user_params['min_transfer_volume'],
    culture_ratio=user_params['culture_factor']
)

Sucess rate: 100.0%
Sucess rate (water): 100.0%


In [17]:
df_volumes.head()

,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Kan,Water
Well,,,,,,,,,,,,,,,,,
A1,30.0,15.0,15.892945,10.0,81.273852,13.811701,39.281637,5.262847,471.906858,15.155637,7.730955,6.011450,23.749257,14.441149,18.600299,5.0,711.881413
B1,30.0,15.0,15.892945,10.0,81.273852,13.811701,39.281637,5.262847,471.906858,15.155637,7.730955,6.011450,23.749257,14.441149,18.600299,5.0,711.881413
C1,30.0,15.0,15.892945,10.0,81.273852,13.811701,39.281637,5.262847,471.906858,15.155637,7.730955,6.011450,23.749257,14.441149,18.600299,5.0,711.881413
D1,30.0,15.0,5.392659,10.0,30.925942,19.107629,85.109990,10.453672,242.473083,232.495019,8.700869,16.696069,13.948046,5.138533,14.383862,5.0,740.174628
E1,30.0,15.0,5.392659,10.0,30.925942,19.107629,85.109990,10.453672,242.473083,232.495019,8.700869,16.696069,13.948046,5.138533,14.383862,5.0,740.174628


In [18]:
df_conc_level.head()

,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Kan
Well,,,,,,,,,,,,,,,,
A1,high,high,high,high,high,high,low,high,high,high,high,high,high,high,high,high
B1,high,high,high,high,high,high,low,high,high,high,high,high,high,high,high,high
C1,high,high,high,high,high,high,low,high,high,high,high,high,high,high,high,high
D1,high,high,high,high,high,high,low,high,high,high,high,high,high,high,low,high
E1,high,high,high,high,high,high,low,high,high,high,high,high,high,high,low,high


Add volumes for culture

In [19]:
df_volumes['Culture'] = user_params['well_volume'] / user_params['culture_factor']
df_volumes.head()

,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Kan,Water,Culture
Well,,,,,,,,,,,,,,,,,,
A1,30.0,15.0,15.892945,10.0,81.273852,13.811701,39.281637,5.262847,471.906858,15.155637,7.730955,6.011450,23.749257,14.441149,18.600299,5.0,711.881413,15.0
B1,30.0,15.0,15.892945,10.0,81.273852,13.811701,39.281637,5.262847,471.906858,15.155637,7.730955,6.011450,23.749257,14.441149,18.600299,5.0,711.881413,15.0
C1,30.0,15.0,15.892945,10.0,81.273852,13.811701,39.281637,5.262847,471.906858,15.155637,7.730955,6.011450,23.749257,14.441149,18.600299,5.0,711.881413,15.0
D1,30.0,15.0,5.392659,10.0,30.925942,19.107629,85.109990,10.453672,242.473083,232.495019,8.700869,16.696069,13.948046,5.138533,14.383862,5.0,740.174628,15.0
E1,30.0,15.0,5.392659,10.0,30.925942,19.107629,85.109990,10.453672,242.473083,232.495019,8.700869,16.696069,13.948046,5.138533,14.383862,5.0,740.174628,15.0


Check if sum of all volumes is equal to total well volume

In [22]:
EPS = 0.000001
assert (np.sum(df_volumes.values, axis=1) - user_params['well_volume'] <= EPS).all(), 'Sum of all volumes is not equal to total well volume!' 

### Save volumes in the destination wells to a file

In [23]:
volumes_file = f"{user_params['output_path']}/dest_volumes.csv"
df_volumes.to_csv(volumes_file)

## Add minimal volumes needed in the source plates

Define the volumes for the 24-well source plate:

In [24]:
well_volume = 9000  # including dead volume
dead_volume = 100


In [25]:
tot_vol_water = np.sum(df_volumes['Water'].values)
print(f'Total volume of water needed: {tot_vol_water:.0f} uL + dead volume')

Total volume of water needed: 34238 uL + dead volume


### High level

In [26]:
df_stock_plate_high['Volume [uL]'] = None

for i in range(len(df_stock_plate_high)-2):
    comp = df_stock_plate_high.iloc[i]['Component']
    stock_level = 'high'
    tot_vol_comp = np.sum(
        df_volumes[df_conc_level[comp]==stock_level][comp].values
    )
    df_stock_plate_high.loc[i, 'Volume [uL]'] = np.round(tot_vol_comp)
    
df_stock_plate_high


,Well,Component,Concentration[mM],Volume [uL]
0,A1,MOPS,2000.0,1440.0
1,B1,Tricine,400.0,720.0
2,C1,H3BO3,2.4,539.0
3,D1,Glucose,3000.0,480.0
4,A2,K2SO4,43.5,2299.0
5,B2,K2HPO4,396.0,571.0
6,C2,NH4Cl,1904.0001,391.0
7,D2,MgCl2,15.6,11356.0
8,A3,NaCl,1500.0,11633.0
9,B3,(NH4)6Mo7O24,0.018,561.0


Are the total volumes smaller than well volume of the plate? If not, assign additional wells.

In [27]:
df_new_wells = pd.DataFrame(columns=df_stock_plate_high.columns)
ind_drop = []
num_all_wells = len(df_stock_plate_high)
for i in range(len(df_stock_plate_high)-2):
    tot_volume = df_stock_plate_high.iloc[i]['Volume [uL]']
    if tot_volume + dead_volume > well_volume:
        ind_drop = ind_drop + [i]
        comp = df_stock_plate_high.iloc[i]['Component']
        conc_level = 'high'
        num_wells_needed = int(np.ceil(tot_volume / well_volume))
        print(f'We need {num_wells_needed} well(s) for {comp} with {conc_level} concentration')
        # Include additional wells needed
        indices = [i]
        indices.extend(range(num_all_wells, num_all_wells+num_wells_needed-1))
        num_all_wells += num_wells_needed
        volumes = []
        volume_left = tot_volume + num_wells_needed*dead_volume
        while volume_left > dead_volume:
            volumes = volumes + [min(volume_left, well_volume)] 
            volume_left = volume_left - volumes[-1]
            
        d = {
             'Component' : pd.Series(comp, index=indices),
             'Concentration[mM]': pd.Series(df_stock_plate_high.iloc[i]['Concentration[mM]'], index =indices),
             'Volume [uL]': pd.Series(volumes, index=indices)}

        df = pd.DataFrame(d)
        df_new_wells = df_new_wells.append(df, ignore_index=False)        

    else:
        df_stock_plate_high.at[i, 'Volume [uL]'] += dead_volume
        print('.', end='')
print('Finished.')

df_stock_plate_high.drop(ind_drop, inplace=True)
df_stock_plate_high = df_stock_plate_high.append(df_new_wells, ignore_index=False)
df_stock_plate_high = df_stock_plate_high.dropna(subset=['Volume [uL]'])
df_stock_plate_high = df_stock_plate_high.sort_index().reset_index(drop=True) 


.......We need 2 well(s) for MgCl2 with high concentration
We need 2 well(s) for NaCl with high concentration
......Finished.


In [28]:
df_stock_plate_high


,Well,Component,Concentration[mM],Volume [uL]
0,A1,MOPS,2000.0,1540.0
1,B1,Tricine,400.0,820.0
2,C1,H3BO3,2.4,639.0
3,D1,Glucose,3000.0,580.0
4,A2,K2SO4,43.5,2399.0
5,B2,K2HPO4,396.0,671.0
6,C2,NH4Cl,1904.0001,491.0
7,NaN,MgCl2,15.6,9000.0
8,NaN,NaCl,1500.0,9000.0
9,B3,(NH4)6Mo7O24,0.018,661.0


Assign well names:

In [29]:
well_rows = 'ABCD'
well_columns = '123456'

In [30]:
num_source_wells = len(df_stock_plate_high) 
    
well_names = [f'{row}{column}' for column in well_columns for row in well_rows]
well_names = well_names[:num_source_wells]

df_stock_plate_high['Well'] = well_names
df_stock_plate_high = df_stock_plate_high.set_index("Well")
df_stock_plate_high

,Component,Concentration[mM],Volume [uL]
Well,,,
A1,MOPS,2000.0,1540.0
B1,Tricine,400.0,820.0
C1,H3BO3,2.4,639.0
D1,Glucose,3000.0,580.0
A2,K2SO4,43.5,2399.0
B2,K2HPO4,396.0,671.0
C2,NH4Cl,1904.0001,491.0
D2,MgCl2,15.6,9000.0
A3,NaCl,1500.0,9000.0


### Low level

In [31]:
df_stock_plate_low['Volume [uL]'] = None

for i in range(len(df_stock_plate_low)):
    comp = df_stock_plate_low.iloc[i]['Component']
    stock_level = 'low'
    tot_vol_comp = np.sum(
        df_volumes[df_conc_level[comp]==stock_level][comp].values
    )
    df_stock_plate_low.iloc[i, df_stock_plate_low.columns.get_loc('Volume [uL]')] = np.round(tot_vol_comp)

df_stock_plate_low

,Well,Component,Concentration[mM],Volume [uL]
0,A1,H3BO3,0.1200,667.0
1,B1,K2SO4,8.7000,33.0
2,C1,K2HPO4,79.2000,149.0
3,D1,(NH4)6Mo7O24,0.0009,308.0
4,A2,CoCl2,0.0090,665.0
5,B2,CuSO4,0.0030,614.0
6,C2,MnSO4,0.0240,383.0
7,D2,ZnSO4,0.0030,625.0


In [32]:
df_new_wells = pd.DataFrame(columns=df_stock_plate_low.columns)
ind_drop = []
num_all_wells = len(df_stock_plate_high)
for i in range(len(df_stock_plate_low)):
    tot_volume = df_stock_plate_low.iloc[i]['Volume [uL]']
    if tot_volume + dead_volume > well_volume:
        ind_drop = ind_drop + [i]
        comp = df_stock_plate_low.iloc[i]['Component']
        conc_level = 'low'
        num_wells_needed = int(np.ceil(tot_volume / well_volume))
        print(f'We need {num_wells_needed} well(s) for {comp} with {conc_level} concentration')
        # Include additional wells needed
        indices = [i]
        indices.extend(range(num_all_wells, num_all_wells+num_wells_needed-1))
        num_all_wells += num_wells_needed
        volumes = []
        volume_left = tot_volume + num_wells_needed*dead_volume
        while volume_left > dead_volume:
            volumes = volumes + [min(volume_left, well_volume)] 
            volume_left = volume_left - volumes[-1]
            
        d = {'Component' : pd.Series(comp, index=indices),
             'Concentration[mM]': pd.Series(df_stock_plate_low.iloc[0]['Concentration[mM]'], index =indices),
             'Volume [uL]': pd.Series(volumes, index=indices)}

        df = pd.DataFrame(d)
        df_new_wells = df_new_wells.append(df, ignore_index=False)        

    else:
        df_stock_plate_low.at[i,'Volume [uL]'] += dead_volume
        print('.', end='')
print('Finished.')

df_stock_plate_low.drop(ind_drop, inplace=True)
df_stock_plate_low = df_stock_plate_low.append(df_new_wells, ignore_index=False)
df_stock_plate_low = df_stock_plate_low.sort_index().reset_index(drop=True) 


........Finished.


Assign well names:

In [33]:
df_stock_plate_low = df_stock_plate_low.set_index("Well")
df_stock_plate_low

,Component,Concentration[mM],Volume [uL]
Well,,,
A1,H3BO3,0.1200,767.0
B1,K2SO4,8.7000,133.0
C1,K2HPO4,79.2000,249.0
D1,(NH4)6Mo7O24,0.0009,408.0
A2,CoCl2,0.0090,765.0
B2,CuSO4,0.0030,714.0
C2,MnSO4,0.0240,483.0
D2,ZnSO4,0.0030,725.0


### Fresh components

In [34]:
df_stock_plate_fresh['Volume [uL]'] = None

comp = 'FeSO4'
for i, stock_level in enumerate(list(['low', 'high'])):
    tot_vol_comp = np.sum(
        df_volumes[df_conc_level[comp]==stock_level][comp].values
    )
    df_stock_plate_fresh.loc[i+1, 'Volume [uL]'] = np.round(tot_vol_comp)

# Culture
tot_vol_culture = np.sum(df_volumes['Culture'].values) 
df_stock_plate_fresh.loc[0, 'Volume [uL]'] = np.round(tot_vol_culture)

df_stock_plate_fresh = df_stock_plate_fresh.set_index("Well")
df_stock_plate_fresh


,Component,Concentration[mM],Volume [uL]
Well,,,
A1,Culture,NaN,720.0
B1,FeSO4,0.3,608.0
C1,FeSO4,6.0,552.0


Save source plate instructions:

In [35]:
stock_plate_file = f"{user_params['output_path']}/24-well_stock_plate_high.csv"
df_stock_plate_high.to_csv(stock_plate_file)

In [36]:
stock_plate_file = f"{user_params['output_path']}/24-well_stock_plate_low.csv"
df_stock_plate_low.to_csv(stock_plate_file)

In [37]:
stock_plate_file = f"{user_params['output_path']}/24-well_stock_plate_fresh.csv"
df_stock_plate_fresh.to_csv(stock_plate_file)

## Define transfers for biomek

In [38]:
column_names = [
    "Source Position",
    "Source Well",
    "Destination Position",
    "Destination Well",
    "Transfer Volume [uL]"
]

### Create water transfers

In [39]:
P20_water = pd.DataFrame(columns=column_names)
P200_water = P20_water.copy()

i = 0  # counter for index

comp = 'Water'
    
for dest_well in df_volumes.index:
    vol = df_volumes.at[dest_well, comp]

    # P20 transfer
    if vol < 30:
        P20_water.at[i, "Destination Well"] = dest_well
        P20_water.at[i, "Transfer Volume [uL]"] = vol
        i += 1

    # P200 transfer
    else:
        # Divide the transfer in parts until whole volume is transfered
        # How many transfers
        num_transfers = int(np.ceil(vol / max_tip_volume))
        transf_vol = vol / num_transfers
        for trans_num in range(num_transfers):
            P200_water.at[i, "Destination Well"] = dest_well
            P200_water.at[i, "Transfer Volume [uL]"] = transf_vol
            i += 1

P20_water["Source Position"] = "P3" 
P20_water["Source Well"] = "A1"  # reservoir plate
P20_water["Destination Position"] = "P2" 
P200_water["Source Position"] = "P3" 
P200_water["Source Well"] = "A1"
P200_water["Destination Position"] = "P2" 

### Create kan transfers

In [40]:
P20_kan = pd.DataFrame(columns=column_names)
P200_kan = P20_kan.copy()

i = 0  # counter for index

comp = 'Kan'
    
for dest_well in df_volumes.index:
    vol = df_volumes.at[dest_well, comp]

    # P20 transfer
    if vol < 30:
        P20_kan.at[i, "Destination Well"] = dest_well
        P20_kan.at[i, "Transfer Volume [uL]"] = vol
        i += 1

    # P200 transfer
    else:
        # Divide the transfer in parts until whole volume is transfered
        # How many transfers
        num_transfers = int(np.ceil(vol / max_tip_volume))
        transf_vol = vol / num_transfers
        for trans_num in range(num_transfers):
            P200_kan.at[i, "Destination Well"] = dest_well
            P200_kan.at[i, "Transfer Volume [uL]"] = transf_vol
            i += 1

P20_kan["Source Position"] = "P1" 
P20_kan["Source Well"] = df_stock_plate_high[
    df_stock_plate_high["Component"]==comp
].index[0]  
P20_kan["Destination Position"] = "P2"
P200_kan["Source Position"] = "P1" 
P200_kan["Source Well"] = df_stock_plate_high[
    df_stock_plate_high["Component"]==comp
].index[0]  
P200_kan["Destination Position"] = "P2"


### Create culture transfers

Note that we assume here the volumes for culture are small enough for p20 pipette, if that is not the case you need to adjust.

In [41]:
P20_culture = pd.DataFrame(columns=column_names)

comp = 'Culture'
vol = df_volumes[comp][0]
    
for i, dest_well in enumerate(df_volumes.index):
    
    P20_culture.at[i, "Destination Well"] = dest_well
    P20_culture.at[i, "Transfer Volume [uL]"] = vol

# DBTL 1
# P20_culture["Source Position"] = "P4" 
# P20_culture["Source Well"] = df_stock_plate_low[
#     df_stock_plate_low["Component"]==comp
# ].index[0]

P20_culture["Source Position"] = "P5" 
P20_culture["Source Well"] = df_stock_plate_fresh[
    df_stock_plate_fresh["Component"]==comp
].index[0]
P20_culture["Destination Position"] = "P2" 


### Create component transfers

Create a column to track the current volume in the wells of the source plate:

In [42]:
df_stock_plate_high['Current Volume'] = well_volume
df_stock_plate_low['Current Volume'] = well_volume
df_stock_plate_fresh['Current Volume'] = well_volume

In [43]:
P20_components = pd.DataFrame(columns=column_names) 
P200_components = P20_components.copy()

In [44]:
i = 0  # counter for index

components = list(df_volumes.columns.drop(['Kan', 'Water', 'Culture']))

for comp in components:
    
    for dest_well in df_volumes.index:
        vol = df_volumes.at[dest_well, comp]
        conc_level = df_conc_level.at[dest_well, comp]
        
        # Find which plate, well we need to use as source
        if comp == 'FeSO4':
            plate_position = "P5"
            df_stock_plate = df_stock_plate_fresh
            source_well = "B1" if conc_level == 'low' else "C1"
        else:
            if conc_level == 'high':
                plate_position = "P1" 
                df_stock_plate = df_stock_plate_high
            else:
                plate_position = "P4"
                df_stock_plate = df_stock_plate_low

            # Find the source well in the stock plate with enough volume 
            # Find all wells with this component
            source_wells = df_stock_plate[
                (df_stock_plate["Component"]==comp)
            ].index

            # Find indices where these wells have volume larger than transfer volume
            ind_vol = np.where(df_stock_plate.loc[source_wells, 'Current Volume']  > vol + dead_volume)[0]
            # Assign as the source well the first one with enough volume
            source_well = source_wells[ind_vol[0]]    
        
        # P20 transfer
        if vol < 30:
            P20_components.at[i, "Source Well"] = source_well
            P20_components.at[i, "Destination Well"] = dest_well
            P20_components.at[i, "Transfer Volume [uL]"] = vol
            P20_components.at[i, "Source Position"] = plate_position
            i += 1

        # P200 transfer
        else:
            # Divide the transfer in parts until whole volume is transfered
            # How many transfers
            num_transfers = int(np.ceil(vol / max_tip_volume))
            transf_vol = vol / num_transfers
            for trans_num in range(num_transfers):
                P200_components.at[i, "Source Well"] = source_well
                P200_components.at[i, "Destination Well"] = dest_well
                P200_components.at[i, "Transfer Volume [uL]"] = transf_vol
                P200_components.at[i, "Source Position"] = plate_position
                i += 1
                    
        # Update the current volume in the source well
        df_stock_plate.loc[source_well, 'Current Volume'] -= vol

P20_components["Destination Position"] = "P2" 
P200_components["Destination Position"] = "P2" 

In [45]:
P20_components.head(5)

,Source Position,Source Well,Destination Position,Destination Well,Transfer Volume [uL]
0,P1,A1,P2,A1,30.0
1,P1,A1,P2,B1,30.0
2,P1,A1,P2,C1,30.0
30,P1,A1,P2,A6,30.0
31,P1,A1,P2,B6,30.0


## Calculate number of transfers and tips needed

In [46]:
num_transf_water_p200 = len(P200_water)
num_transf_water_p20 = len(P20_water)
num_transf_comp_p20 = len(P20_components)
num_transf_comp_p200 = len(P200_components)
num_transf_culture = len(P20_culture)
num_transf_kan_p20 = len(P20_kan)
num_transf_kan_p200 = len(P200_kan)
num_mixing = 1  # mixing the culture well 
num_total_transf = num_transf_water_p200 + num_transf_water_p20 + num_transf_comp_p20 + num_transf_comp_p200+num_transf_kan_p20+num_transf_kan_p200+num_transf_culture+num_mixing
print(f'Number of transfers:')
print(f'\t Water (p200): {num_transf_water_p200}')
print(f'\t Water (p20): {num_transf_water_p20}')
print(f'\t Kan (p20): {num_transf_kan_p20}')
print(f'\t Kan (p200): {num_transf_kan_p200}')
print(f'\t Components (p200): {num_transf_comp_p200}')
print(f'\t Components (p20): {num_transf_comp_p20}')
print(f'\t Culture (p20): {num_transf_culture}')
print(f'\t Mixing (p20): {num_mixing}')
print(f'Total number of transfers: {num_total_transf}')

Number of transfers:
	 Water (p200): 204
	 Water (p20): 0
	 Kan (p20): 48
	 Kan (p200): 0
	 Components (p200): 300
	 Components (p20): 498
	 Culture (p20): 48
	 Mixing (p20): 1
Total number of transfers: 1099


Calculate number of tip boxes needed:

In [47]:
print(f'This protocol requires:')
# For water transfers (from a reservoir plate) biomek is using only 7 probes
num_tips_200 = 7 if num_transf_water_p200 > 0 else 0
num_tips_200 += num_transf_comp_p200 + num_transf_kan_p200 
num_tips_20 = 7 if num_transf_water_p20 > 0 else 0
num_tips_20 += num_transf_comp_p20 + num_transf_kan_p20 + num_mixing + num_transf_culture
full_box, rest = divmod(num_tips_200, 96)
print(f'\t{np.ceil(num_tips_200 / 96):.0f} box(es) of p200 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')
full_box, rest = divmod(num_tips_20, 96)
print(f'\t{np.ceil((num_tips_20) / 96):.0f} box(es) of p20 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')


This protocol requires:
	4 box(es) of p200 tips
		3 box(es) + 19 tips
	7 box(es) of p20 tips
		6 box(es) + 19 tips


Introduce a pause in the biomek method after p200 components transfers.

In [48]:
print(f'The first part of the protocol requires:')
# For water transfers (from a reservoir plate) biomek is using only 7 probes
num_tips_200 = 7 if num_transf_water_p200 > 0 else 0
num_tips_200 += num_transf_comp_p200 + num_transf_kan_p200 
num_tips_20 = 7 if num_transf_water_p20 > 0 else 0
num_tips_20 += num_transf_kan_p20 

full_box, rest = divmod(num_tips_200, 96)
print(f'\t{np.ceil(num_tips_200 / 96):.0f} box(es) of p200 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')
full_box, rest = divmod(num_tips_20, 96)
print(f'\t{np.ceil((num_tips_20) / 96):.0f} box(es) of p20 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')

The first part of the protocol requires:
	4 box(es) of p200 tips
		3 box(es) + 19 tips
	1 box(es) of p20 tips
		0 box(es) + 48 tips


For the next step we need:

In [49]:
print(f'The second part of the protocol requires:')
# For water transfers (from a reservoir plate) biomek is using only 7 probes
num_tips_20 = num_transf_comp_p20  + num_mixing + num_transf_culture
full_box, rest = divmod(num_tips_20, 96)
print(f'\t{np.ceil((num_tips_20) / 96):.0f} box(es) of p20 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')

The second part of the protocol requires:
	6 box(es) of p20 tips
		5 box(es) + 67 tips


### Save biomek files

In [50]:
biomek_files_dir = f"{user_params['output_path']}/biomek_files/"
os.makedirs(biomek_files_dir, exist_ok=True)

P200_water_file = f"{biomek_files_dir}/P200_water.csv"
P20_water_file = f"{biomek_files_dir}/P20_water.csv"
P20_kan_file = f"{biomek_files_dir}/P20_kan.csv"
P200_components_file = f"{biomek_files_dir}/P200_components.csv"
P20_components_file = f"{biomek_files_dir}/P20_components.csv"
P20_culture_file = f"{biomek_files_dir}/P20_culture.csv"

P200_water.to_csv(P200_water_file, index=False)
P20_water.to_csv(P20_water_file, index=False)
P20_kan.to_csv(P20_kan_file, index=False)
P200_components.to_csv(P200_components_file, index=False)
P20_components.to_csv(P20_components_file, index=False)
P20_culture.to_csv(P20_culture_file, index=False)